# Membership Inference Attack Analysis Examples

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/feat/116-testable-notebooks/notebooks/membership_inference_attack_analysis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/feat/116-testable-notebooks/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/feat/116-testable-notebooks/notebooks/membership_inference_attack_analysis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/feat/116-testable-notebooks/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

In this notebook, we want to show you how to use the `privacy-evaluator` tool to perform a Membership Inference Attack Analysis on either a PyTorch or Tensorflow model. As an example we will conduct the Membership Inference Black Box Rule Based Attack and apply an in-depth analysis on this method. In order to get a more general introduction into the different kinds of Membership Inference Attacks, please see [membership_inference_attack.ipynb](https://github.com/privML/privacy-evaluator/blob/feat/116-testable-notebooks/notebooks/membership_inference_attack.ipynb).

## Setup

First, you should set the notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator@feat/116-testable-notebooks

In [1]:
import numpy as np
import torch

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti 
from privacy_evaluator.datasets.torch.cifar10 import TorchCIFAR10
from privacy_evaluator.classifiers.classifier import Classifier

from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis
from privacy_evaluator.attacks.membership_inference import MembershipInferenceBlackBoxRuleBasedAttack

from privacy_evaluator.attacks.membership_inference.data_structures.attack_input_data import AttackInputData
from privacy_evaluator.attacks.membership_inference.data_structures.slicing import Slicing

## Conduct Membership Inference Attack Analysis

Now we can start with conducting the Membership Inference Attacks Analysis. Therefore, we prepared two similar paths: one for the PyTorch model and one for the TensorFlow model. For both paths, we use simple neural networks trained on the CIFAR-10 dataset and implement a Lightweight Deep Convolutional Neural Network architecture (For more details, please read the following paper: https://www.scitepress.org/Papers/2018/67520/67520.pdf).

### PyTorch

We start the analysis with the PyTorch model.

#### Load CIFAR10 Dataset

Before we can start to conduct the membership inference attack analysis, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the PyTorch model.

In [2]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TorchCIFAR10.numpy()

0it [00:00, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `torch.nn.CrossEntropyLoss`), the number of classes and the input shape of our CIFAR-10 dataset.

In [3]:
# Initalize PyTorch model as a Classifier
target_model = Classifier(
    torch_dcti.load_dcti(), # PyTorch DCTI 
    loss=torch.nn.CrossEntropyLoss(reduction="none"), # Loss function of the PyTorch model
    nb_classes=TorchCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TorchCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

In [4]:
attack_analysis = MembershipInferenceAttackAnalysis(
    MembershipInferenceBlackBoxRuleBasedAttack, 
    AttackInputData(
        x_train, 
        y_train, 
        x_test, 
        y_test
    )
)

slicing = Slicing(
    entire_dataset=True, 
    by_class=True, 
    by_classification_correctness=True
)

result = attack_analysis.analyse(
    target_model, 
    np.concatenate((x_train, x_test)), 
    np.concatenate((y_train, y_test)), 
    np.concatenate((np.ones(len(x_train)), np.zeros(len(x_test)))), 
    slicing
)

RuntimeError: CUDA out of memory. Tried to allocate 704.00 MiB (GPU 0; 7.77 GiB total capacity; 1.40 GiB already allocated; 220.44 MiB free; 1.42 GiB reserved in total by PyTorch)

In [ ]:
print("\n".join((str(r) for r in result)))